--- Day 11: Seating System ---

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

* If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
* If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
* Otherwise, the seat's state does not change.

Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

After a second round, the seats with four or more occupied adjacent seats become empty again:

```
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```

This process continues for three more rounds:

```
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##

#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##

#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?


In [1]:
#!fsharp
let testInput="""L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL"""

let taskInput = System.IO.File.ReadAllText "task.txt"

In [1]:
#!fsharp
let parse (input:string) =
    input.Replace("\r", "").Split("\n")

let print (grid: string[]) =
    let asText = Array.reduce (fun acc row -> acc + "\n" + row ) grid
    asText

parse testInput |> print

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

In [1]:
#!fsharp
let getTile x y (grid: string[]) =
    if y < 0 || y >= grid.Length || x < 0 then
        '.'
    else
        let row = grid.[y]
        if  x >= row.Length then
            '.'
        else
            row.[x]

let getNeighbourCount x y (grid: string[]) =
    let mutable sum = 0
    for dy in -1..1 do
        for dx in -1..1 do
            if ( dx <> 0 || dy <> 0 ) && getTile (x+dx) (y+dy) grid = '#' then
                sum <- sum + 1
                0
            else 
                0
            |> ignore
    sum                
            


In [1]:
#!fsharp
let simulateStep (grid: string[]) =
    grid
    |> Array.mapi 
        (fun y row ->
            row
            |> String.mapi  
                (fun x tile -> 
                    if tile = 'L' && getNeighbourCount x y grid = 0 then
                        '#'
                    elif tile = '#' && getNeighbourCount x y grid >= 4 then
                        'L'
                    else
                        tile

                )
        )

testInput |> parse |> simulateStep |> simulateStep |>  print

#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##

In [1]:
#!fsharp
let rec simulate (grid: string[]) =
    let newGrid = simulateStep grid
    if newGrid = grid then
        grid
    else
        simulate newGrid

testInput |> parse |> simulate |> print

#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##

In [1]:
#!fsharp
let countPeople (grid:string[]) =
    grid
    |> Array.sumBy 
        ( fun row -> 
            row
            |> Seq.sumBy ( fun tile -> if tile = '#' then 1 else 0 ) 
        )

testInput |> parse |> simulate |> countPeople

37

In [1]:
#!fsharp
taskInput |> parse |> simulate |> countPeople

2178

--- Part Two ---

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

```
.............
.L.L.#.#.#.#.
.............
```

The empty seat below would see no occupied seats:

```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#

#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?


In [1]:
#!fsharp
let zeroMatrix (grid: string[]) =
    grid
    |> Array.map 
        ( fun row -> 
            Array.zeroCreate<int> row.Length
        )


let spreadVisibility (sx, sy) (dx, dy) (grid:string[]) (seats: byref<int[][]>) =
    let mutable x = sx
    let mutable y = sy
    let mutable seesPerson = false
    while x >= 0 && y >= 0 && y < grid.Length && x < grid.[y].Length do
        if seesPerson then
            seats.[y].[x] <- seats.[y].[x] + 1
        seesPerson <- match getTile x y grid with
                      | 'L' -> false
                      | '#' -> true
                      | _ -> seesPerson
        x <- x + dx
        y <- y + dy
    seats

let calculateVisibility (grid: string[]) = 
    let mutable seats = zeroMatrix grid
    let width = grid.[0].Length
    let height = grid.Length
    for y in 0 .. height - 1 do
        // E
        spreadVisibility (0, y) (1, 0) grid &seats |> ignore
        // W
        spreadVisibility (width-1, y) (-1, 0) grid &seats |> ignore
        // SE
        spreadVisibility (0, y) (1, 1) grid &seats |> ignore
        // NW
        spreadVisibility (width-1, y) (-1, -1) grid &seats |> ignore
        // NE
        spreadVisibility (0, y) (1, -1) grid &seats |> ignore
        // SW
        spreadVisibility (width-1, y) (-1, 1) grid &seats |> ignore

    for x in 0 .. width - 1 do
        // S
        spreadVisibility (x, 0) (0, 1) grid &seats |> ignore
        // N
        spreadVisibility (x, height-1) (0, -1) grid &seats |> ignore
        if x > 0 then
            // SE
            spreadVisibility (x, 0) (1, 1) grid &seats |> ignore
            // NE
            spreadVisibility (x, height-1) (1, -1) grid &seats |> ignore
        if x < width - 1 then
            // NW
            spreadVisibility (x, height-1)  (-1, -1) grid &seats |> ignore
            // SW
            spreadVisibility (x, 0) (-1, 1) grid &seats |> ignore
    
    // printfn "%s" (print grid)
    seats

testInput |> parse |> simulateStep |> calculateVisibility

index,value
0,"[ 3, 5, 5, 5, 5, 5, 5, 5, 5, 3 ]"
1,"[ 4, 7, 7, 7, 7, 7, 7, 7, 6, 5 ]"
2,"[ 5, 8, 8, 8, 8, 8, 8, 6, 7, 5 ]"
3,"[ 5, 8, 8, 8, 8, 8, 8, 8, 7, 5 ]"
4,"[ 5, 8, 7, 8, 8, 8, 8, 8, 8, 5 ]"
5,"[ 5, 7, 8, 8, 8, 8, 8, 8, 7, 5 ]"
6,"[ 5, 7, 7, 8, 7, 7, 7, 7, 7, 5 ]"
7,"[ 5, 6, 8, 8, 8, 8, 8, 8, 7, 4 ]"
8,"[ 4, 7, 7, 8, 8, 8, 7, 7, 7, 5 ]"
9,"[ 3, 5, 5, 5, 5, 5, 5, 5, 5, 3 ]"


In [1]:
#!fsharp
let simulateStep2 (grid: string[]) =
    let visibility = calculateVisibility grid
    grid
    |> Array.mapi 
        (fun y row ->
            row
            |> String.mapi  
                (fun x tile -> 
                    if tile = 'L' && visibility.[y].[x] = 0 then
                        '#'
                    elif tile = '#' && visibility.[y].[x] >= 5 then
                        'L'
                    else
                        tile

                )
        )

testInput 
|> parse 
|> simulateStep2 
|> simulateStep2 
|> simulateStep2 
|> simulateStep2 
|> simulateStep2 
|> simulateStep2 
|> print

#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#

In [1]:
#!fsharp
let rec simulate2 (grid: string[]) =
    let newGrid = simulateStep2 grid
    if newGrid = grid then
        grid
    else
        simulate2 newGrid

In [1]:
#!fsharp
testInput |> parse |> simulate2 |> countPeople

26

In [1]:
#!fsharp
taskInput |> parse |> simulate2 |> countPeople

1978